# Background Correction Notebook

<div class="custom-button-row">
    <a 
        class="custom-button custom-download-button" href="../../notebooks/07_measurement_and_quantification/background_correction_notebook.ipynb" download>
        <i class="fas fa-download"></i> Download this Notebook
    </a>
    <a
    class="custom-button custom-download-button" href="https://colab.research.google.com/github/HMS-IAC/bobiac/blob/gh-pages/colab_notebooks/07_measurement_and_quantification/background_correction_notebook.ipynb" target="_blank">
        <img class="button-icon" src="../../_static/logo/icon-google-colab.svg" alt="Open in Colab">
        Open in Colab
    </a>
</div>

In [ ]:
# /// script
# requires-python = ">=3.10"
# dependencies = [
#     "matplotlib",
#     "numpy",
#     "scikit-image",
#     "scipy",
#     "tifffile",
#     "imagecodecs",
#     "pooch",
# ]
# ///

 ## Overview

 In this notebook, we will explore different approaches to **background correction** in fluorescence microscopy images. Background correction is a crucial pre-processing step that helps remove unwanted background signal and improves the quality of quantitative analysis. We will use the [**scikit-image**](https://scikit-image.org/docs/stable/) library to perform the background correction. Background substraction is useful when the background is uniform and the signal to noise ratio is high.

 We will demonstrate a simple background subtraction method using a sample fluorescence image. The main approaches we'll cover are:

 - Subtracting a constant background value (e.g. mode or median of the image)
 - Selecting and averaging background regions to determine background level

 The choice of method depends on your specific imaging conditions and the nature of the background in your images. Here we'll demonstrate a basic approach that works well for images with relatively uniform background and distinct fluorescent signals.

 <p class="alert alert-info">
     <strong>Note:</strong> Background correction should be done on raw images before any other processing steps. The corrected images can then be used for further analysis like segmentation and quantification.
 </p>

## Importing libraries

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import skimage
import scipy
import pooch

## Load and Display Images

Let's load and display with `matplotlib` our raw and segmented images (labeled images):

In [ ]:
# raw image and labeled mask
image = skimage.io.imread("../../_static/images/quant/WF_drosophila_100x_140_w1Widefield FITC0063.tif")

# Display results
plt.figure(figsize=(8, 4))
plt.imshow(image, cmap="gray")
plt.title("Original")
plt.axis("off")
plt.tight_layout()
plt.show()

print(f"Image min: {image.min():.3f}, max: {image.max():.3f}, mean: {image.mean():.3f}")

## Background substraction: mode substraction 

Background substraction can be done in different ways. If the background dominates the image (e.g. dark field microscopy), the most common pixel value (the mode) can serve as a rough background estimate. We'll subtract this value from the image. 

First, we compute the mode of the image:

In [ ]:
# Flatten image and get the mode
mode_val = scipy.stats.mode(image.ravel(), keepdims=False).mode
print(f"Estimated background (mode): {mode_val:.3f}")

Then, we subtract the mode from the image and clip the result to be between 0 and 1:

In [ ]:
image_mode_sub = image - mode_val

Finally, we can visualize the background-corrected image:

In [ ]:
plt.figure(figsize=(10, 8))
plt.subplot(121)
plt.imshow(image, cmap='gray')
plt.title("Original")
plt.axis('off')

plt.subplot(122)
plt.imshow(image_mode_sub, cmap='gray')
plt.title("Background substracted (mode)")
plt.axis('off')

plt.tight_layout()
plt.show()

## Background substraction: rolling ball algorithm

The rolling ball algorithm is a method for background substraction that uses a [rolling ball](https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_rolling_ball.html) to estimate the background. It is a good method for background substraction when the background is not uniform. The radius parameter configures how distant pixels should be taken into account for determining the background intensity. 

Therefore, we can estimate the background by using the rolling ball algorithm:


In [ ]:
background_residue = skimage.restoration.rolling_ball(image, radius=100)

plt.figure(figsize=(8, 4))
plt.imshow(background_residue, cmap='gray')
plt.title("Background (rolling ball)")
plt.axis('off')
plt.show()

Afterwards, we subtract the background residue from the image:

In [ ]:

image_rb_sub = image - background_residue

plt.figure(figsize=(10, 10))

plt.subplot(131)
plt.imshow(image, cmap='gray')
plt.title("Original")
plt.axis('off')

plt.subplot(132)
plt.imshow(background_residue, cmap='gray')
plt.title("Background (rolling ball)")
plt.axis('off')

plt.subplot(133)
plt.imshow(image_rb_sub, cmap='gray')
plt.title("Background substracted (rolling ball)")
plt.axis('off')

plt.tight_layout()
plt.show()

## Background substraction: selected regions

Sometimes the background isn't uniform, or the mode isn't representative. In these cases, we can manually choose a region we believe contains only background and estimate the average intensity in that region.

First, we select a region of the image that we believe contains only background, and we compute the average intensity in that region:

In [ ]:
# Choose a top-left corner patch assumed to be background
bg_patch = image[0:150, 0:150]
bg_mean = np.mean(bg_patch)
print(f"Estimated background (mean of selected region): {bg_mean:.3f}")

As we did before, we can substract this value from the image, and clip the result to be between 0 and 1:

In [ ]:
image_bgmean_sub = image - bg_mean

And visualize the result:

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(121)
plt.imshow(image, cmap='gray')
plt.title("Original")
plt.axis('off')

plt.subplot(122)
plt.imshow(image_bgmean_sub, cmap='gray')
plt.title("Background substracted (regions)")
plt.axis('off')

plt.tight_layout()
plt.show()

### ✍️ Exercise: Find a selected region of the image that is representative of the background

In this exercise, let's try to:
- find a selected region of the image that is representative of the background
- compute the mean intensity of the selected region
- plot the image and the selected region

In this case, using a selected region to estimate the background does not work well. This could be due to the fact that the background is not uniform, or that the region is not representative of the background; therefore, the rolling ball algorithm proves to be a better choice.

## Other Background Subtraction Techniques

These are more advanced or specialized techniques you can explore:

- **Morphological opening**: Removes small foreground objects to approximate the background.
- **Gaussian/median filtering**: Smooths out the image to isolate large-scale variations.
- **Rolling ball algorithm**: Emulates a ball "rolling" under the image to estimate the background.
- **Polynomial surface fitting**: Useful when background varies gradually across the field.
- **Tiled/local background subtraction**: Estimate and subtract background patch-by-patch.

Your method choice should depend on image modality, signal-to-noise, and application.

A good reference for background correction is the [scikit-image documentation](https://scikit-image.org/docs/0.25.x/api/skimage.restoration.html).
